In [1]:
from requests_oauthlib import OAuth2Session
import socket
import webbrowser
import os
import requests
import base64
import random
import BaseHTTPServer
import hashlib
import json
import nacl.signing
import urlparse
import re
#import NetworkManager
import uuid

In [2]:
eduvpn_base_uri = 'https://static.eduvpn.nl/'
eduvpn_key = 'E5On0JTtyUVZmcWd+I/FXRm32nSq8R2ioyW7dcu/U88='

In [3]:
landing_page = """<html>
<head>
<title>eduvpn</title>
</head>
<body>
<h1>You can now close this window</h1>
</body>
</html>
"""
def get_open_port():
        """find an unused port"""
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("",0))
        s.listen(1)
        port = s.getsockname()[1]
        s.close()
        return port
    
def gen_code_verifier(length=128):
    choices = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-._~'
    r = random.SystemRandom()
    return "".join(r.choice(choices) for _ in range(length))
    
def gen_code_challenge(code_verifier):  
    return base64.urlsafe_b64encode(hashlib.sha256(code_verifier).digest()).rstrip('=')

def one_request(port):
    # listen for one request
    class RequestHandler(BaseHTTPServer.BaseHTTPRequestHandler):
        def do_GET(self):
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.end_headers()
            self.wfile.write(landing_page)
            self.server.path = self.path

    httpd = BaseHTTPServer.HTTPServer(('', port), RequestHandler)
    httpd.handle_request()
    httpd.server_close()
    parsed = urlparse.urlparse(httpd.path)
    return urlparse.parse_qs(parsed.query)

def configurator(text):
    for line in text.split('\n'):
        split = line.split('#')[0].strip().split()
        if len(split) == 0:
            continue
        if len(split) == 1:
            yield (split[0], None)
        elif len(split) == 2:
            yield split
        else:
            yield (split[0], split[1:])
            
def parse_ovpn(configtext):
    config = {}
    for tag in 'ca', 'tls-auth', 'cert', 'key':
        x = re.search('<{}>(.*)</{}>'.format(tag, tag), configtext, flags=re.S)
        if x:
            full_match = x.group(0)
            config[tag] = x.group(1).replace('\r\n', '\n')
            configtext = configtext.replace(full_match, '')
    config.update(dict(configurator(configtext)))
    return config

In [4]:
# used for verifying signatures
verify_key = nacl.signing.VerifyKey(eduvpn_key, encoder=nacl.encoding.Base64Encoder)

In [5]:
# retrieve a list of instances
inst_doc_url = eduvpn_base_uri + '/instances.json'
inst_doc_sig_url = eduvpn_base_uri + '/instances.json.sig'
inst_doc = requests.get(inst_doc_url)
inst_doc_sig = requests.get(inst_doc_sig_url)

# verify signature
_ = verify_key.verify(smessage=inst_doc.content, signature=inst_doc_sig.content.decode('base64'))

instances = [i['base_uri'] for i in inst_doc.json()['instances']]
instances

[u'https://nl.eduvpn.org/',
 u'https://esciencecenter.eduvpn.nl/',
 u'https://ru.eduvpn.nl/',
 u'https://surf.eduvpn.nl/',
 u'https://ut.eduvpn.nl/',
 u'https://demo.eduvpn.nl/']

In [6]:
instance = 'https://demo.eduvpn.nl/'

In [7]:
# get info from server
info = requests.get(instance + '/info.json')
#info_sig = requests.get(instance + '/info.json.sig')
#_ = verify_key.verify(smessage=info.content, signature=info_sig.content.decode('base64'))

In [8]:
instance_urls = info.json()['api']['http://eduvpn.org/api#2']
authorization_endpoint = instance_urls['authorization_endpoint']
token_endpoint = instance_urls['token_endpoint']
api_base_uri = instance_urls['api_base_uri']

In [9]:
port = get_open_port()
client_id = "org.eduvpn.app"
redirect_uri = 'http://127.0.0.1:%s/callback' % port
auth_uri = 'https://demo.eduvpn.nl/portal/_oauth/authorize'
response_type = "code"
scope = "config"
code_challenge_method = "S256"
code_verifier = gen_code_verifier()
code_challenge = gen_code_challenge(code_verifier)

In [10]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=[scope])
# we add extra arguments below for PKCE
authorization_url, state = oauth.authorization_url(authorization_endpoint, code_challenge_method=code_challenge_method,
                                                   code_challenge=code_challenge)

In [11]:
webbrowser.open(authorization_url)
response = one_request(port)

127.0.0.1 - - [17/Jul/2017 11:14:01] "GET /callback?code=TKeyzzpITSu0C1Oh3ZMdsryaqTuXTObJ%2FrvBQapu%2FnB1vQKq9GjqUyBxJa%2FJMP%2BJfPIcWF0KLEVPtKcrRfniD3sidHlwZSI6ImF1dGhvcml6YXRpb25fY29kZSIsImF1dGhfa2V5IjoiYTc2MWU3OGY3ZmExYWEyMDIzYThlY2MyMmY4ODY0NjQiLCJ1c2VyX2lkIjoiZTExMzdlZjc0MmFkZjgzMDhhYWI3ZTBhNmI2NDNjOGUyMmNiZjNmYiIsImNsaWVudF9pZCI6Im9yZy5lZHV2cG4uYXBwIiwic2NvcGUiOiJjb25maWciLCJyZWRpcmVjdF91cmkiOiJodHRwOlwvXC8xMjcuMC4wLjE6NTAzOTBcL2NhbGxiYWNrIiwiY29kZV9jaGFsbGVuZ2UiOiJkVXBjLWFGNUxONnNiYWg4aEFoVkhLN1BQNlJ3RlVQbm9EMUdqNVdNVk4wIiwiZXhwaXJlc19hdCI6IjIwMTctMDctMTcgMDg6MTk6MDEifQ%3D%3D&state=5m2sxDbd28UCrB659vqEfYdsHkqvn8 HTTP/1.1" 200 -


In [12]:
token = oauth.fetch_token(token_endpoint, code=response['code'][0], code_verifier=code_verifier)

In [13]:
#profile_list = oauth.get(api_base_uri + '/profile_list').content

In [14]:
#user_info = oauth.get(api_base_uri + '/user_info').content

In [15]:
#user_messages = oauth.get(api_base_uri + '/user_messages').content

In [16]:
#system_messages = oauth.get(api_base_uri + '/system_messages').content

In [17]:
create_config = oauth.post(api_base_uri + '/create_config', data={'display_name': 'notebook', 'profile_id': 'internet'})

In [18]:
create_keypair = oauth.post(api_base_uri + '/create_keypair', data={'display_name': 'notebook'})

In [19]:
keypair = json.loads(create_keypair.content)['create_keypair']['data']
cert = keypair['certificate']
key = keypair['private_key']

In [20]:
profile_config = oauth.get(api_base_uri + '/profile_config?profile_id=internet').content

In [23]:
config = parse_ovpn(profile_config)
#config.update({'cert': cert, 'key': key})

In [22]:
settings = {'connection': {'id': 'eduvpn',
  'type': 'vpn',
  'uuid': str(uuid.uuid4())},
 'ipv4': {
  'method': 'auto',
},
 'ipv6': {
  'method': 'auto',
},
 'vpn': {'data': {'auth': config.get('auth', 'SHA256'),
   'ca': config['ca'],
   'cert': config['cert'],
   'cipher': config.get('cipher', 'AES-256-CBC'),
   'comp-lzo': config.get('auth', 'adaptive'),
   'connection-type': config.get('connection-type', 'tls'),
   'dev': 'tun',
   'key': config['key'],
   'remote': ":".join(config['remote']),
   'remote-cert-tls': 'server',
   'ta': config['tls-auth'],
   'ta-dir': config.get('key-direction', '1'),
   'tls-cipher': config.get('tls-cipher','TLS-ECDHE-RSA-WITH-AES-256-GCM-SHA384')},
  'service-type': 'org.freedesktop.NetworkManager.openvpn'}
}

In [24]:
config

{'auth': 'SHA256',
 'ca': '\n-----BEGIN CERTIFICATE-----\nMIIFJDCCAwygAwIBAgIJAJ1NwjmG+n/3MA0GCSqGSIb3DQEBCwUAMBExDzANBgNV\nBAMTBlZQTiBDQTAeFw0xNzAyMjcxNDA5NTJaFw0yMjAyMjcxNDA5NTJaMBExDzAN\nBgNVBAMTBlZQTiBDQTCCAiIwDQYJKoZIhvcNAQEBBQADggIPADCCAgoCggIBALOW\n8ZrqXnNnqaojP90UkGIXbXLl7U8kborvKF/gQm6gsk3O0obhTVDdEj3kYKFGxmqu\nQHJo3C2amro4wVIv/BhXzIv6oP11Va/6FcwAYj8disNcxJgXWSUdARrEIC3PME58\nLd+a2jvXL/m8xBp+qVThvfmq2Pak1olhbgDueUSsQYiQZMq2rX3/1f+SInxZt3kJ\nZak2nNf7uZ1+XaoR78gkBHqszXlaFdPByC2bQn0vrsGurUHTXO843MDfsckbeNEE\n0LWOOSBzGwYnjfGmm/JzM06jHq+vyfrFwl28uNuCVRwJpvmjJC8+O8TYFgOrx8Oe\nQ7gbC+Wrx1Nv/u2j59ofTPk2R5oNdrYt3h5YVv0WocHmiYb+CzdwxBvyZ50Uy1+x\n8CKV1gYtjPBc6OOMNYyXfXlu0qtkLljV/Pdt4hxUNyyptPvJgP8xLg+z+RoZZmxV\n0XEZpfYV4x6m1/8sEqYANx+5U+3KXZznUXlevyLqgS5tn3hGuxwRuxb5AifwRRmF\nz5wSVuDilCGnzNGyqJvP4rKAlGakQx8nyFIsdMsa7OpQVd4n+ex5HhfMlSa7Bvpj\nDK+8p3ynd11jihCkFAzOqldBN2hR1gCjx2b52zLk0vzTyyiCEebqb9hqKf+jb3/j\nFyE9gzlm2uR9rvvyVKnsDnuCb5vsnWdPTUvBov6tAgMBAAGjfzB9MB0GA1UdDgQW\nBBTmp4VYXwUkBWl7cK